In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import jax
import jax.numpy as jnp
import jax.nn as jnn
from jax import jit,grad,vmap,value_and_grad,jacfwd,jacrev, random
from jax.example_libraries.optimizers import adam
from functools import partial
import numpy as np
import numpy.random as npr
import math
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import sklearn.metrics
from jaxopt import LBFGS

initializer = jnn.initializers.glorot_normal()

def init_glorot_params(layer_sizes, key = random.PRNGKey(10)):
    return [(initializer(key, (m, n), jnp.float32), jnp.zeros(n))
          for m, n, in zip(layer_sizes[:-1], layer_sizes[1:])]

# A MLP for first domain
def f(params, x, y):
    inputs = jnp.concatenate((x,y), axis=-1)
    for w, b in params:
        outputs = jnp.dot(inputs, w) + b
        inputs = jnn.tanh(outputs)
    return outputs  

# Hyperparameters
layer_size = [2, 60, 60, 60, 1]
step_size = 1e-3
train_iters = 99001
train_iters_bfgs = 100

# Initial guess of parameters
params_fwd = init_glorot_params(layer_size)

# Domain parameters for training
xmin = 0
xmax = 4
ymin = 0
ymax = 4
nx = 70
ny = 70

Xi = jnp.linspace(xmin,xmax,nx)
Yi = jnp.linspace(ymin,ymax,ny)
xp,yp = jnp.meshgrid(Xi,Yi)
X = jnp.stack([xp.flatten(),yp.flatten()],1)

x = X[:, 0].reshape(-1,1)
y = X[:, 1].reshape(-1,1)

a = np.ones([nx*ny,2])
b = np.ones([nx*ny,2])
z = 0
q = 0
for i in X:
    if i[1] <= 0.25:
        a[z, 0] = i[0]
        a[z, 1] = i[1]
        z += 1
    else:
        b[q, 0] = i[0]
        b[q, 1] = i[1]
        q += 1
inputs_source = a[0:z,:]
x_source = inputs_source[:,0].reshape(-1,1)
y_source = inputs_source[:,1].reshape(-1,1)
inputs_outside = b[0:q,:]
x_outside = inputs_outside[:,0].reshape(-1,1)
y_outside = inputs_outside[:,1].reshape(-1,1)
   
# Getting coordinates for boundaries of domain separately
L = jnp.stack([jnp.ones(ny)*xmin,Yi],1)
R = jnp.stack([jnp.ones(ny)*xmax,Yi],1)
B = jnp.stack([Xi,jnp.ones(nx)*ymin],1)
T = jnp.stack([Xi,jnp.ones(nx)*ymax],1)

t_x = T[:,0].reshape(-1,1)
t_y = T[:,1].reshape(-1,1)
b_x = B[:,0].reshape(-1,1)
b_y = B[:,1].reshape(-1,1)
l_x = L[:,0].reshape(-1,1)
l_y = L[:,1].reshape(-1,1)
r_x = R[:,0].reshape(-1,1)
r_y = R[:,1].reshape(-1,1)

# Ploting input points ouside injection region
plt.figure(figsize=(5,5))
plt.scatter(x_outside,y_outside,color='green',marker='.',label='')
# Ploting input points ouside injection region
plt.scatter(x_source,y_source,color='orange',marker='.',label='')
#plotting left boundry
plt.scatter(l_x,l_y,color='red',marker='o',label='')
#plotting right boundry
plt.scatter(r_x,r_y,color='red',marker='o',label='')
#plotting top boundry
plt.scatter(t_x,t_y,color='red',marker='o',label='')
#plotting bottom boundry
plt.scatter(b_x,b_y,color='red',marker='o',label='')
plt.title('Collocation points')


BC_l_n = 0
BC_r_n = 0
BC_t = 0
BC_b_n = 0
q_m = -100
k_left = k_approx(l_x,l_y).reshape(-1,1)
k_right = k_approx(r_x,r_y).reshape(-1,1)
k_bottom = k_approx(b_x,b_y).reshape(-1,1)

def objective(params_fwd):
    
    # For domain outside source region
    # Define the function p(x, y)
    p = lambda x, y: f(params_fwd, x, y)
    
    # First derivatives of p (lambda functions)
    p_x = lambda x, y: jacfwd(p, argnums=0)(x, y)
    p_y = lambda x, y: jacfwd(p, argnums=1)(x, y)
    
    # Multiply by k after first derivatives (lambda functions)
    k_p_x = lambda x, y: k_approx(x,y) * p_x(x, y)
    k_p_y = lambda x, y: k_approx(x,y) * p_y(x, y)
    
    # Second derivatives of (k * p_x) and (k * p_y) (lambda functions)
    p_k_xx = lambda x, y: jacfwd(k_p_x, argnums=0)(x, y)
    p_k_yy = lambda x, y: jacfwd(k_p_y, argnums=1)(x, y)
    
    # Boundary conditions (now using lambda functions for p_x, k(x, y), etc.)
    bc_top = (vmap(p)(t_x, t_y).reshape(-1,1) - BC_t)**2
    bc_bottom = (-k_bottom * vmap(p_y)(b_x, b_y).reshape(-1,1) - BC_b_n)**2
    bc_left = (-k_left * vmap(p_x)(l_x, l_y).reshape(-1,1) - BC_l_n)**2
    bc_right = (k_right * vmap(p_x)(r_x, r_y).reshape(-1,1) - BC_r_n)**2
    
    # Compute boundary loss
    loss_bc = jnp.mean(bc_top) + jnp.mean(bc_left) + jnp.mean(bc_right) + jnp.mean(bc_bottom)
    
    # Compute the equation loss using second derivatives of (k * p_x) and (k * p_y)
    eq = (vmap(p_k_xx)(x_outside, y_outside).reshape(-1,1) + vmap(p_k_yy)(x_outside, y_outside).reshape(-1,1))**2
    eq_source = (vmap(p_k_xx)(x_source, y_source).reshape(-1,1) + vmap(p_k_yy)(x_source, y_source).reshape(-1,1) - q_m)**2
    loss_eq = jnp.mean(eq) + jnp.mean(eq_source)
    
    # Total loss
    LOSS = loss_bc + loss_eq
    
    return LOSS

# Adam optimizer
@jit
def resnet_update(params_fwd,opt_state):
    """ Compute the gradient for a batch and update the parameters """
    value, grads = value_and_grad(objective)(params_fwd)
    opt_state = opt_update(0, grads, opt_state)
    return get_params(opt_state), opt_state, value
@jit
def resnet_update_bfgs(params):
    """ Compute the gradient for a batch and update the parameters using bfgs """
    solver = LBFGS(fun=objective, value_and_grad=False, has_aux=False, maxiter=500, tol=1e-6, stepsize=0.0)
    res = solver.run(params)
    #params, opt_state = res
    return res.params

opt_init, opt_update, get_params = adam(step_size, b1=0.9, b2=0.999, eps=1e-08)
opt_state = opt_init(params_fwd)


for i in range(train_iters):
    params_fwd, opt_state, value = resnet_update(params_fwd,opt_state)
    if i % 1000 == 0:
        print("Iteration {0:3d} objective {1}".format(i,objective(params_fwd)))       
        
for i in range(train_iters_bfgs):
    params_fwd = resnet_update_bfgs(params_fwd)
    if i % 10 == 0:
        print("Iteration {0:3d} objective {1}".format(i,objective(params_fwd)))

# Testing grid generation
xminR = 0
xmaxR = 4
yminR = 0
ymaxR = 4
nxR = 45
nyR = 45
xR = jnp.linspace(xminR,xmaxR,nxR)
yR = jnp.linspace(yminR,ymaxR,nyR)
xpR,ypR = jnp.meshgrid(xR,yR)
XR = jnp.stack([xpR.flatten(),ypR.flatten()],1)
xR = XR[:,0].reshape(-1,1)
yR = XR[:,1].reshape(-1,1)

a1 = np.ones([nxR*nyR,2])
b1 = np.ones([nxR*nyR,2])
z1 = 0
q1 = 0
for i in XR:
    if i[1] <= 0.25:
        a1[z1, 0] = i[0]
        a1[z1, 1] = i[1]
        z1 += 1
    else:
        b1[q1, 0] = i[0]
        b1[q1, 1] = i[1]
        q1 += 1
R_source = a1[0:z1,:]
xR_source = R_source[:,0].reshape(-1,1)
yR_source = R_source[:,1].reshape(-1,1)
R_outside = b1[0:q1,:]
xR_outside = R_outside[:,0].reshape(-1,1)
yR_outside = R_outside[:,1].reshape(-1,1)

# Ploting test points
plt.figure(figsize=(5,5))
plt.scatter(xR_source,yR_source,color='green',marker='.',label='')
plt.scatter(xR_outside,yR_outside,color='gold',marker='.',label='')
plt.title('Testing points')

# Getting I-PINNs based solution for pressure
p_approx = lambda x,y: f(params_fwd,x,y)
P_approx = vmap(p_approx)(xR_outside,yR_outside).reshape(-1,1)
P_s_approx = vmap(p_approx)(xR_source,yR_source).reshape(-1,1)

P_approx = np.concatenate((P_approx,P_s_approx),0).reshape(-1,1)
xR = np.concatenate((xR_outside,xR_source), 0).reshape(-1,1)
yR = np.concatenate((yR_outside,yR_source), 0).reshape(-1,1)

# Getting I-PINNs based solution for velocity
v_x_approx = lambda x, y: -k_approx(x,y) * jacfwd(p_approx,0)(x, y)
v_y_approx = lambda x, y: -k_approx(x,y) * jacfwd(p_approx,1)(x, y)
V_approx =  np.sqrt(np.square(vmap(v_x_approx)(xR_outside,yR_outside).reshape(-1,1)) + np.square(vmap(v_y_approx)(xR_outside,yR_outside).reshape(-1,1)))
V_s_approx = np.sqrt(np.square(vmap(v_x_approx)(xR_source,yR_source).reshape(-1,1)) + np.square(vmap(v_y_approx)(xR_source,yR_source).reshape(-1,1)))

V_approx = np.concatenate((V_approx,V_s_approx),0).reshape(-1,1)
xR = np.concatenate((xR_outside,xR_source), 0).reshape(-1,1)
yR = np.concatenate((yR_outside,yR_source), 0).reshape(-1,1)

#Extacting COMSOL data for pressure
COMSOL_P_data = np.loadtxt('/kaggle/input/problem16-comsol-p1/problem16_COMSOL_P1.txt')
L = COMSOL_P_data.shape[0]
xc_P = COMSOL_P_data[:,0].reshape(-1,1)
yc_P = COMSOL_P_data[:,1].reshape(-1,1)

a2 = np.ones([L,3])
b2 = np.ones([L,3])
z2 = 0
q2 = 0
for i in COMSOL_P_data:
    if i[1] <= 0.25:
        a2[z2, 0] = i[0]
        a2[z2, 1] = i[1]
        a2[z2, 2] = i[2]
        z2 += 1
    else:
        b2[q2, 0] = i[0]
        b2[q2, 1] = i[1]
        b2[q2, 2] = i[2]
        q2 += 1
c_p_source = a2[0:z2,:]
xc_P_source = c_p_source[:,0].reshape(-1,1)
yc_P_source = c_p_source[:,1].reshape(-1,1)
COMSOL_P_source = c_p_source[:,2].reshape(-1,1)
c_p_outside = b2[0:q2,:]
xc_P_outside = c_p_outside[:,0].reshape(-1,1)
yc_P_outside = c_p_outside[:,1].reshape(-1,1)
COMSOL_P_outside = c_p_outside[:,2].reshape(-1,1) 


# Getting I-PINNs based solution at points where COMSOL solution is available
pc_approx = lambda x,y: f(params_fwd,x,y)
Pc_approx = vmap(pc_approx)(xc_P_outside,yc_P_outside).reshape(-1,1)
Pc_s_approx = vmap(pc_approx)(xc_P_source,yc_P_source).reshape(-1,1)

Pc_approx = np.concatenate((Pc_approx,Pc_s_approx),0).reshape(-1,1)
COMSOL_P = np.concatenate((COMSOL_P_outside,COMSOL_P_source),0).reshape(-1,1)
INPUTS_xc = np.concatenate((xc_P_outside,xc_P_source), 0).reshape(-1,1)
INPUTS_yc = np.concatenate((yc_P_outside,yc_P_source), 0).reshape(-1,1)

def RMSE(actual, predicted):
    MSE = jnp.square(jnp.subtract(actual,predicted)).mean()
    return math.sqrt(MSE)
print("Root Mean Square Error for P = ", str(RMSE(COMSOL_P,Pc_approx)))

#Extacting COMSOL data for velocity
COMSOL_V_data = np.loadtxt('/kaggle/input/problem16-comsol-v1/problem16_COMSOL_V1.txt')
L = COMSOL_V_data.shape[0]
xc_V = COMSOL_V_data[:,0].reshape(-1,1)
yc_V = COMSOL_V_data[:,1].reshape(-1,1)

a3 = np.ones([L,3])
b3 = np.ones([L,3])
z3 = 0
q3 = 0
for i in COMSOL_V_data:
    if i[1] <= 0.25:
        a3[z3, 0] = i[0]
        a3[z3, 1] = i[1]
        a3[z3, 2] = i[2]
        z3 += 1
    else:
        b3[q3, 0] = i[0]
        b3[q3, 1] = i[1]
        b3[q3, 2] = i[2]
        q3 += 1
c_v_source = a3[0:z3,:]
xc_V_source = c_v_source[:,0].reshape(-1,1)
yc_V_source = c_v_source[:,1].reshape(-1,1)
COMSOL_V_source = c_v_source[:,2].reshape(-1,1)
c_v_outside = b3[0:q3,:]
xc_V_outside = c_v_outside[:,0].reshape(-1,1)
yc_V_outside = c_v_outside[:,1].reshape(-1,1)
COMSOL_V_outside = c_v_outside[:,2].reshape(-1,1) 


# Getting I-PINNs based solution at points where COMSOL solution is available
vc_x_approx = lambda x, y: -k_approx(x,y) * jacfwd(pc_approx,0)(x, y)
vc_y_approx = lambda x, y: -k_approx(x,y) * jacfwd(pc_approx,1)(x, y)
Vc_approx = np.sqrt(np.square(vmap(vc_x_approx)(xc_V_outside,yc_V_outside).reshape(-1,1)) + np.square(vmap(vc_y_approx)(xc_V_outside,yc_V_outside).reshape(-1,1)))
Vc_s_approx = np.sqrt(np.square(vmap(vc_x_approx)(xc_V_source,yc_V_source).reshape(-1,1)) + np.square(vmap(vc_y_approx)(xc_V_source,yc_V_source).reshape(-1,1)))

Vc_approx = np.concatenate((Vc_approx,Vc_s_approx),0).reshape(-1,1)
COMSOL_V = np.concatenate((COMSOL_V_outside,COMSOL_V_source),0).reshape(-1,1)
INPUTS_V_xc = np.concatenate((xc_V_outside,xc_V_source), 0).reshape(-1,1)
INPUTS_V_yc = np.concatenate((yc_V_outside,yc_V_source), 0).reshape(-1,1)

def RMSE(actual, predicted):
    MSE = jnp.square(jnp.subtract(actual,predicted)).mean()
    return math.sqrt(MSE)
print("Root Mean Square Error for V = ", str(RMSE(COMSOL_V,Vc_approx)))

cmap = plt.get_cmap('rainbow')
fontsize = 15

# Plotting AdaI-PINNs solution of pressure
plt.figure()
ax = plt.subplot()
ax.set_aspect('equal', adjustable='box')
im = plt.tricontourf(xR.ravel(), yR.ravel(), P_approx.ravel(), 100, cmap=cmap)  # Flattening xR, yR
ax.tick_params(axis='both', labelsize=fontsize)
plt.title("Ada-PINNs solution for P ", fontsize=fontsize)
plt.xlabel("X", fontsize=fontsize)
plt.ylabel("Y", fontsize=fontsize)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, cax=cax)
cbar.ax.tick_params(labelsize=fontsize)  # Set font size for colorbar labels
plt.savefig('pinns_u_inv', bbox_inches='tight', dpi=1200)
plt.show()

# Plotting AdaI-PINNs solution of velocity
plt.figure()
ax = plt.subplot()
ax.set_aspect('equal', adjustable='box')
im = plt.tricontourf(xR.ravel(), yR.ravel(), V_approx.ravel(), 100, cmap=cmap)  # Flattening xR, yR
ax.tick_params(axis='both', labelsize=fontsize)
plt.title("Ada-PINNs solution for V ", fontsize=fontsize)
plt.xlabel("X", fontsize=fontsize)
plt.ylabel("Y", fontsize=fontsize)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, cax=cax)
cbar.ax.tick_params(labelsize=fontsize)  # Set font size for colorbar labels
plt.savefig('pinns_u_inv', bbox_inches='tight', dpi=1200)
plt.show()

# Plotting COMSOL solution of pressure
plt.figure()
ax = plt.subplot()
ax.set_aspect('equal', adjustable='box')
im = plt.tricontourf(INPUTS_xc.ravel(), INPUTS_yc.ravel(), COMSOL_P.ravel(), 100, cmap=cmap)  # Flattening xc, yc
ax.tick_params(axis='both', labelsize=fontsize)
plt.title("COMSOL solution for P ", fontsize=fontsize)
plt.xlabel("X", fontsize=fontsize)
plt.ylabel("Y", fontsize=fontsize)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, cax=cax)
cbar.ax.tick_params(labelsize=fontsize)  # Set font size for colorbar labels
plt.savefig('pinns_u_inv', bbox_inches='tight', dpi=1200)
plt.show()

# Plotting COMSOL solution of velocity
plt.figure()
ax = plt.subplot()
ax.set_aspect('equal', adjustable='box')
im = plt.tricontourf(INPUTS_V_xc.ravel(), INPUTS_V_yc.ravel(), COMSOL_V.ravel(), 100, cmap=cmap)  # Flattening xc, yc
ax.tick_params(axis='both', labelsize=fontsize)
plt.title("COMSOL solution for V ", fontsize=fontsize)
plt.xlabel("X", fontsize=fontsize)
plt.ylabel("Y", fontsize=fontsize)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, cax=cax)
cbar.ax.tick_params(labelsize=fontsize)  # Set font size for colorbar labels
plt.savefig('pinns_u_inv', bbox_inches='tight', dpi=1200)
plt.show()

# Plotting AdaI-PINNs solution at COMSOL data points of pressure
plt.figure()
ax = plt.subplot()
ax.set_aspect('equal', adjustable='box')
im = plt.tricontourf(INPUTS_xc.ravel(), INPUTS_yc.ravel(), Pc_approx.ravel(), 100, cmap=cmap)  # Flattening xc, yc
ax.tick_params(axis='both', labelsize=fontsize)
plt.title("AdaI-PINNs solution at COMSOL data points for P ", fontsize=fontsize)
plt.xlabel("X", fontsize=fontsize)
plt.ylabel("Y", fontsize=fontsize)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, cax=cax)
cbar.ax.tick_params(labelsize=fontsize)  # Set font size for colorbar labels
plt.savefig('pinns_u_inv', bbox_inches='tight', dpi=1200)
plt.show()

# Plotting AdaI-PINNs solution at COMSOL data points of velocity
plt.figure()
ax = plt.subplot()
ax.set_aspect('equal', adjustable='box')
im = plt.tricontourf(INPUTS_V_xc.ravel(), INPUTS_V_yc.ravel(), Vc_approx.ravel(), 100, cmap=cmap)  # Flattening xc, yc
ax.tick_params(axis='both', labelsize=fontsize)
plt.title("AdaI-PINNs solution at COMSOL data points for V ", fontsize=fontsize)
plt.xlabel("X", fontsize=fontsize)
plt.ylabel("Y", fontsize=fontsize)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, cax=cax)
cbar.ax.tick_params(labelsize=fontsize)  # Set font size for colorbar labels
plt.savefig('pinns_u_inv', bbox_inches='tight', dpi=1200)
plt.show()